In [ ]:
import v20
import pandas as pd
import numpy as np
import json
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import os
import mplfinance as mpf
import openai
import base64
import requests
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# OANDA API configuration using environment variables
api = v20.Context(
    hostname='api-fxpractice.oanda.com',
    port=443,
    ssl=True,
    token=os.getenv('OANDA_API_TOKEN'),
    datetime_format='RFC3339'
)
access_token = os.getenv('OANDA_API_TOKEN')

# Set OpenAI API key from environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')




# Step 4: Fetch Historical Data
def fetch_forex_data(from_date, to_date, granularity, instrument):
    client = API(access_token=access_token, environment="practice")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = client.request(request)
            if response:
                for candle in response.get('candles'):
                    rec = {
                        'time': candle.get('time')[0:19],
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        print(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)

prices = fetch_forex_data('2016-01-01T00:00:00Z', '2018-02-18T00:00:00Z', 'M15', 'EUR_USD')
def simple_moving_average_strategy(df):
    df['SMA20'] = df['close'].rolling(window=20).mean()
    df['SMA50'] = df['close'].rolling(window=50).mean()
    df['signal'] = 0
    df.loc[50:, 'signal'] = np.where(df['SMA20'][50:] > df['SMA50'][50:], 1, 0)
    df['position'] = df['signal'].diff()
    return df

df = simple_moving_average_strategy(prices)


import os
import pandas as pd
import mplfinance as mpf

# Load the dataset
# prices = pd.read_csv('EURUSD_M15.csv')

# Ensure the 'time' column is in datetime format
prices['time'] = pd.to_datetime(prices['time'])

# Set the 'time' column as the index
prices.set_index('time', inplace=True)

# Define the window size and step size
window_size = 1000  # Number of ticks in each window
step_size = 500     # Number of ticks to move the window

# Create directories based on forex pair, timeframe, and year
pair = 'EUR_USD'
timeframe = 'M15'
base_dir = f'{pair}/{timeframe}'

# Create base directory if it does not exist
os.makedirs(base_dir, exist_ok=True)

def plot_candlestick_chart(df, filename):
    # Ensure the date formatting is similar to the sample
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'

    # Define the style of the chart
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')

    # Add moving averages if they exist
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    
    # Plot the chart
    mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, savefig=filename)

# Function to create moving window chart images
def create_moving_window_charts(prices, window_size, step_size, base_dir):
    start_index = 0
    end_index = window_size
    chart_count = 0

    while start_index < len(prices):
        # Get the window data
        window_data = prices.iloc[start_index:end_index].copy()
        
        # Create the directory structure based on the year
        year = window_data.index[0].year
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)

        # Create the chart image filename
        start_date = window_data.index[0].strftime('%Y-%m-%d_%H-%M')
        end_date = window_data.index[-1].strftime('%Y-%m-%d_%H-%M')
        filename = os.path.join(year_dir, f'{start_date}_to_{end_date}.png')

        # Plot and save the chart image
        plot_candlestick_chart(window_data, filename)
        
        # Print the created chart image path
        print(f'Created chart image: {filename}')
        
        # Move the window
        start_index += step_size
        end_index = start_index + window_size

# Create and analyze moving window charts
create_moving_window_charts(prices, window_size, step_size, base_dir)


import os
import pandas as pd
import mplfinance as mpf

# Load the dataset
# prices = pd.read_csv('EURUSD_M15.csv')

# Ensure the 'time' column is in datetime format
prices['time'] = pd.to_datetime(prices['time'])

# Set the 'time' column as the index
prices.set_index('time', inplace=True)

# Define the window size and step size
window_size = 1000  # Number of ticks in each window
step_size = 500     # Number of ticks to move the window

# Create directories based on forex pair, timeframe, and year
pair = 'EUR_USD'
timeframe = 'M15'
base_dir = f'{pair}/{timeframe}'

# Create base directory if it does not exist
os.makedirs(base_dir, exist_ok=True)

def plot_candlestick_chart(df, filename):
    # Ensure the date formatting is similar to the sample
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'

    # Define the style of the chart
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')

    # Add moving averages if they exist
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    
    # Plot the chart
    mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, savefig=filename)

# Function to create moving window chart images
def create_moving_window_charts(prices, window_size, step_size, base_dir):
    start_index = 0
    end_index = window_size
    chart_count = 0

    while start_index < len(prices):
        # Get the window data
        window_data = prices.iloc[start_index:end_index].copy()
        
        # Create the directory structure based on the year
        year = window_data.index[0].year
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)

        # Create the chart image filename
        start_date = window_data.index[0].strftime('%Y-%m-%d_%H-%M')
        end_date = window_data.index[-1].strftime('%Y-%m-%d_%H-%M')
        filename = os.path.join(year_dir, f'{start_date}_to_{end_date}.png')

        # Plot and save the chart image
        plot_candlestick_chart(window_data, filename)
        
        # Print the created chart image path
        print(f'Created chart image: {filename}')
        
        # Move the window
        start_index += step_size
        end_index = start_index + window_size

# Create and analyze moving window charts
create_moving_window_charts(prices, window_size, step_size, base_dir)
